# Collaboration and Competition

---

You are welcome to use this coding environment to train your agent for the project.  Follow the instructions below to get started!

### 1. Start the Environment

Run the next code cell to install a few packages.  This line will take a few minutes to run!

In [3]:
!pip -q install ./python

^C


In [ ]:
!pip install dataclasses
!pip install matplotlib

The environment is already saved in the Workspace and can be accessed at the file path provided below. 

In [ ]:
from unityagents import UnityEnvironment
import numpy as np

env = UnityEnvironment(file_name="/data/Tennis_Linux_NoVis/Tennis")

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [ ]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

In [ ]:
import os
import torch as T
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt

from dataclasses import dataclass

### 2. Examine the State and Action Spaces

Run the code cell below to print some information about the environment.

In [ ]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Note that **in this coding environment, you will not be able to watch the agents while they are training**, and you should set `train_mode=True` to restart the environment.

In [ ]:
#for i in range(5):                                         # play game for 5 episodes
#    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
#    states = env_info.vector_observations                  # get the current state (for each agent)
#    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
#    while True:
#        actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
#        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
#        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
#        next_states = env_info.vector_observations         # get next state (for each agent)
#        rewards = env_info.rewards                         # get reward (for each agent)
#        dones = env_info.local_done                        # see if episode finished
#        scores += env_info.rewards                         # update the score (for each agent)
#        states = next_states                               # roll over states to next time step
#        if np.any(dones):                                  # exit loop if episode finished
#            break
#    print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

When finished, you can close the environment.

In [ ]:
#env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  A few **important notes**:
- When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```
- To structure your work, you're welcome to work directly in this Jupyter notebook, or you might like to start over with a new file!  You can see the list of files in the workspace by clicking on **_Jupyter_** in the top left corner of the notebook.
- In this coding environment, you will not be able to watch the agents while they are training.  However, **_after training the agents_**, you can download the saved model weights to watch the agents on your own machine! 

In [ ]:
def plot_scores(scores, output_file=None):
    episodes = len(scores)
    x = np.arange(1, episodes + 1)

    plt.figure(figsize=(10, 6))
    plt.plot(x, scores, linewidth=2, marker='o', markersize=6, linestyle='-')

    plt.title("Agent's Performance over Episodes", fontsize=18, fontweight='bold')
    plt.xlabel('Episodes', fontsize=14, fontweight='bold')
    plt.ylabel('Scores', fontsize=14, fontweight='bold')

    plt.grid(True, linestyle='--', linewidth=0.5)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)

    if output_file:
        plt.savefig(output_file, dpi=300, bbox_inches='tight')
    else:
        plt.show()

In [ ]:
class SharedReplayBuffer:
    def __init__(self, max_size, critic_dims, actor_dims, 
            n_actions, n_agents, batch_size):
        self.mem_size = max_size
        self.mem_cntr = 0
        self.n_agents = n_agents
        self.actor_dims = actor_dims
        self.critic_dims = critic_dims
        self.batch_size = batch_size
        self.n_actions = n_actions

        self.state_memory = np.zeros((self.mem_size, self.critic_dims), dtype=np.float32)
        self.new_state_memory = np.zeros((self.mem_size, self.critic_dims), dtype=np.float32)
        self.reward_memory = np.zeros((self.mem_size, self.n_agents), dtype=np.float32)
        self.terminal_memory = np.zeros((self.mem_size, self.n_agents), dtype=np.bool)

        self.init_actor_memory()

    def init_actor_memory(self):
        self.actor_state_memory = []
        self.actor_new_state_memory = []
        self.actor_action_memory = []

        for i in range(self.n_agents):
            self.actor_state_memory.append(
                            np.zeros((self.mem_size, self.actor_dims[i])))
            self.actor_new_state_memory.append(
                            np.zeros((self.mem_size, self.actor_dims[i])))
            self.actor_action_memory.append(
                            np.zeros((self.mem_size, self.n_actions)))


    def store_transition(self, raw_obs, state, action, reward, 
                               raw_obs_, state_, done):
        
        index = self.mem_cntr % self.mem_size

        for agent_idx in range(self.n_agents):
            self.actor_state_memory[agent_idx][index] = raw_obs[agent_idx]
            self.actor_new_state_memory[agent_idx][index] = raw_obs_[agent_idx]
            self.actor_action_memory[agent_idx][index] = action[agent_idx]

        self.state_memory[index] = state
        self.new_state_memory[index] = state_
        self.reward_memory[index] = reward
        self.terminal_memory[index] = done
        self.mem_cntr += 1

    def sample_buffer(self):
        max_mem = min(self.mem_cntr, self.mem_size)

        batch = np.random.choice(max_mem, self.batch_size, replace=False)

        states = self.state_memory[batch]
        rewards = self.reward_memory[batch]
        states_ = self.new_state_memory[batch]
        terminal = self.terminal_memory[batch]

        actor_states = []
        actor_new_states = []
        actions = []
        for agent_idx in range(self.n_agents):
            actor_states.append(self.actor_state_memory[agent_idx][batch])
            actor_new_states.append(self.actor_new_state_memory[agent_idx][batch])
            actions.append(self.actor_action_memory[agent_idx][batch])

        return actor_states, states, actions, rewards, \
               actor_new_states, states_, terminal

    def ready(self):
        if self.mem_cntr >= self.batch_size:
            return True
        return False

In [ ]:
class CriticNetwork(nn.Module):
    def __init__(self, critic_lr, input_dims, fc1_dims, fc2_dims, 
                    n_agents, n_actions):
        super().__init__()
        self.critic_lr = critic_lr
        self.input_dims = input_dims
        self.fc1_dims = fc1_dims
        self.fc2_dims = fc2_dims
        self.n_actions = n_actions
        self.n_agents = n_agents
        self.fc1 = nn.Linear(self.input_dims+self.n_agents*self.n_actions, self.fc1_dims)
        self.fc2 = nn.Linear(self.fc1_dims, self.fc2_dims)
        self.q = nn.Linear(self.fc2_dims, 1)
        
        nn.init.xavier_uniform_(self.fc1.weight)
        nn.init.xavier_uniform_(self.fc2.weight)
        nn.init.xavier_uniform_(self.q.weight)
        
        self.fc1.bias.data.zero_()
        self.fc2.bias.data.zero_()
        self.q.bias.data.zero_()

        self.optimizer = optim.Adam(self.parameters(), lr=self.critic_lr)
        self.device = T.device('cuda:0' if T.cuda.is_available() else 'cpu')
 
        self.to(self.device)

    def forward(self, state, action):
        x = F.relu(self.fc1(T.cat([state, action], dim=1)))
        x = F.relu(self.fc2(x))
        q = self.q(x)

        return q

In [ ]:
class ActorNetwork(nn.Module):
    def __init__(self, actor_lr, input_dims, fc1_dims, fc2_dims, 
                 n_actions):
        super().__init__()
        self.actor_lr = actor_lr
        self.input_dims = input_dims
        self.fc1_dims = fc1_dims
        self.fc2_dims = fc2_dims
        self.n_actions = n_actions
        self.fc1 = nn.Linear(self.input_dims, self.fc1_dims)
        self.fc2 = nn.Linear(self.fc1_dims, self.fc2_dims)
        self.mu = nn.Linear(self.fc2_dims, self.n_actions)
        
        nn.init.xavier_uniform_(self.fc1.weight)
        nn.init.xavier_uniform_(self.fc2.weight)
        nn.init.xavier_uniform_(self.mu.weight)
        
        self.fc1.bias.data.zero_()
        self.fc2.bias.data.zero_()
        self.mu.bias.data.zero_()

        self.optimizer = optim.Adam(self.parameters(), lr=self.actor_lr)
        self.device = T.device('cuda:0' if T.cuda.is_available() else 'cpu')
 
        self.to(self.device)

    def forward(self, state):
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        mu = F.tanh(self.mu(x))

        return mu

In [ ]:
class OUActionNoise:
    def __init__(self, mu=0.0, sigma=0.15, theta=0.2, dt=1e-2, x0=None):
        self.theta = theta
        self.mu = mu
        self.sigma = sigma
        self.dt = dt
        self.x0 = x0
        self.reset()

    def __call__(self):
        x = self.x_prev + self.theta * (self.mu - self.x_prev) * self.dt + \
                self.sigma * np.sqrt(self.dt) * np.random.normal(size=self.mu.shape)
        self.x_prev = x

        return x

    def reset(self):
        self.x_prev = self.x0 if self.x0 is not None else np.zeros_like(self.mu)

In [ ]:
class Agent:
    def __init__(self, actor_dims, critic_dims, n_actions, n_agents, agent_idx,
                    actor_lr, critic_lr, fc1, 
                    fc2, gamma, tau, epsilon, epsilon_decay):
        
        self.actor_dims = actor_dims
        self.critic_dims = critic_dims
        self.n_actions = n_actions
        self.n_agents = n_agents
        self.agent_idx = agent_idx
        self.actor_lr = actor_lr
        self.critic_lr = critic_lr
        self.gamma = gamma
        self.tau = tau
        self.fc1 = fc1
        self.fc2 = fc2
        self.agent_name = 'agent_%s' % self.agent_idx
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        
        self.actor = ActorNetwork(self.actor_lr, self.actor_dims, self.fc1, self.fc2, self.n_actions)
        self.critic = CriticNetwork(self.critic_lr, self.critic_dims, self.fc1, self.fc2, self.n_agents, self.n_actions)
        self.target_actor = ActorNetwork(self.actor_lr, self.actor_dims, self.fc1, self.fc2, self.n_actions)
        self.target_critic = CriticNetwork(self.critic_lr, self.critic_dims, self.fc1, self.fc2, self.n_agents, self.n_actions)
        
        self.noise = OUActionNoise(mu=np.zeros(self.n_actions))

        self.update_network_parameters(tau=1)
        
    def save(self, model_prefix):
        T.save(
        {
            'local_actor': self.actor.state_dict(),
            'target_actor': self.target_actor.state_dict(),
            'local_critic': self.critic.state_dict(),
            'target_critic': self.target_critic.state_dict(),
        },
        f'./{model_prefix}.pt')

    def choose_action(self, observation):
        self.actor.eval()
        state = T.tensor([observation], dtype=T.float).to(self.actor.device)
        mu = self.actor.forward(state)
        
        if np.random.rand() < self.epsilon:
            mu_prime = mu + T.tensor(self.noise(), dtype=T.float).to(self.actor.device)
        else:
            mu_prime = mu
            
        self.actor.train()
            
        self.epsilon *= self.epsilon_decay

        return mu_prime.detach().cpu().numpy()[0]

    def update_network_parameters(self, tau=None):
        if tau is None:
            tau = self.tau

        target_actor_params = self.target_actor.named_parameters()
        actor_params = self.actor.named_parameters()

        target_actor_state_dict = dict(target_actor_params)
        actor_state_dict = dict(actor_params)
        for name in actor_state_dict:
            actor_state_dict[name] = tau*actor_state_dict[name].clone() + \
                    (1-tau)*target_actor_state_dict[name].clone()

        self.target_actor.load_state_dict(actor_state_dict)

        target_critic_params = self.target_critic.named_parameters()
        critic_params = self.critic.named_parameters()

        target_critic_state_dict = dict(target_critic_params)
        critic_state_dict = dict(critic_params)
        for name in critic_state_dict:
            critic_state_dict[name] = tau*critic_state_dict[name].clone() + \
                    (1-tau)*target_critic_state_dict[name].clone()

        self.target_critic.load_state_dict(critic_state_dict)

In [ ]:
class MADDPG:
    def __init__(self, config):
        self.actor_dims = config.actor_dims
        self.critic_dims = config.critic_dims
        self.n_agents = config.n_agents
        self.n_actions = config.n_actions
        self.scenario = config.scenario
        self.actor_lr = config.actor_lr
        self.critic_lr = config.critic_lr
        self.fc1 = config.fc1_dims
        self.fc2 = config.fc2_dims
        self.gamma = config.gamma
        self.tau = config.tau
        self.epsilon = config.epsilon
        self.epsilon_decay = config.epsilon_decay
        self.agents = []
        
        for agent_idx in range(self.n_agents):
            self.agents.append(Agent(self.actor_dims[agent_idx], self.critic_dims,  
                            self.n_actions, self.n_agents, agent_idx,
                            actor_lr=self.actor_lr, critic_lr=self.critic_lr, 
                            fc1=self.fc1, fc2=self.fc2, gamma=self.gamma, tau=self.tau,
                            epsilon=self.epsilon, epsilon_decay=self.epsilon_decay))

    def choose_action(self, raw_obs):
        actions = []
        for agent_idx, agent in enumerate(self.agents):
            action = agent.choose_action(raw_obs[agent_idx])
            actions.append(action)
        return actions

    def learn(self, memory):
        if not memory.ready():
            print('GATHERING DATA')
            return
        
        actor_states, states, actions, rewards, \
        actor_new_states, states_, dones = memory.sample_buffer()

        device = self.agents[0].actor.device

        states = T.tensor(states, dtype=T.float).to(device)
        actions = T.tensor(actions, dtype=T.float).to(device)
        rewards = T.tensor(rewards, dtype=T.float).to(device)
        states_ = T.tensor(states_, dtype=T.float).to(device)
        dones = T.tensor(dones.astype(np.uint8)).to(device)

        all_agents_new_actions = []
        all_agents_new_mu_actions = []
        old_agents_actions = []

        for agent_idx, agent in enumerate(self.agents):
            new_states = T.tensor(actor_new_states[agent_idx], 
                                 dtype=T.float).to(device)

            new_pi = agent.target_actor.forward(new_states)

            all_agents_new_actions.append(new_pi)
            mu_states = T.tensor(actor_states[agent_idx], 
                                 dtype=T.float).to(device)
            pi = agent.actor.forward(mu_states)
            all_agents_new_mu_actions.append(pi)
            old_agents_actions.append(actions[agent_idx])

        new_actions = T.cat([acts for acts in all_agents_new_actions], dim=1)
        mu = T.cat([acts for acts in all_agents_new_mu_actions], dim=1)
        old_actions = T.cat([acts for acts in old_agents_actions],dim=1)

        for agent_idx, agent in enumerate(self.agents):
            #critic_value_ = agent.target_critic.forward(states_, new_actions).flatten() # torch 0.4.0 :(
            critic_value_ = agent.target_critic.forward(states_, new_actions).view(-1)
            critic_value_[dones[:,0]] = 0.0
            #critic_value = agent.critic.forward(states, old_actions).flatten() # torch 0.4.0 :(
            critic_value = agent.critic.forward(states, old_actions).view(-1)

            target = rewards[:,agent_idx] + agent.gamma*critic_value_
            critic_loss = F.mse_loss(target, critic_value)
            agent.critic.optimizer.zero_grad()
            critic_loss.backward(retain_graph=True)
            agent.critic.optimizer.step()

            #actor_loss = agent.critic.forward(states, mu).flatten() # torch 0.4.0 :(
            actor_loss = agent.critic.forward(states, mu).view(-1)
            actor_loss = -T.mean(actor_loss)
            agent.actor.optimizer.zero_grad()
            actor_loss.backward(retain_graph=True)
            agent.actor.optimizer.step()

            agent.update_network_parameters()

In [ ]:
@dataclass
class PipelineConfig:
    n_agents: int
    actor_dims: int
    critic_dims: int
    actor_lr: float
    critic_lr: float
    tau: float
    n_actions: int
    input_dims: tuple
    gamma: float
    mem_size: int
    batch_size: float
    fc1_dims: int
    fc2_dims: int
    scenario: str
    epsilon: float
    epsilon_decay: float

In [ ]:
def create_state_vector(observation):
    state = np.array([])
    for obs in observation:
        state = np.concatenate([state, obs])
    return state

In [ ]:
def fetch_actor_and_critic_obs_dims(observation, n_agents):
    actor_dims = []
    for i in range(n_agents):
        actor_dims.append(observation[i].shape[0])
    critic_dims = sum(actor_dims)
    return actor_dims, critic_dims

In [ ]:
def launcher(env, actions, episodes=100, model_prefix='model'):
    brain_name = env.brain_names[0]
    env_info = env.reset(train_mode=True)[brain_name]
    observation_shape = env_info.vector_observations[0].shape
    n_agents = len(env_info.agents)
    actor_dims, critic_dims = fetch_actor_and_critic_obs_dims(env_info.vector_observations, n_agents)

    pipeline_config = PipelineConfig(
        n_agents=n_agents,
        actor_dims=actor_dims,
        critic_dims=critic_dims,
        actor_lr=1e-4,
        critic_lr=1e-3,
        tau=1e-3,
        n_actions=actions,
        input_dims=tuple(observation_shape),
        gamma=0.99,
        mem_size=int(1e6),
        batch_size=256,
        fc1_dims=128,
        fc2_dims=128,
        scenario='competition',
        epsilon=1.0,
        epsilon_decay=1e-6,
    )

    scores = []
    best_score = -np.inf
    global_timestep = 0
    maddpg = MADDPG(config=pipeline_config)
    memory = SharedReplayBuffer(pipeline_config.mem_size, pipeline_config.critic_dims, pipeline_config.actor_dims, 
                    pipeline_config.n_actions, pipeline_config.n_agents, pipeline_config.batch_size) 

    for episode in range(episodes):
        score = 0
        dones = [False] * n_agents
        env_info = env.reset(train_mode=True)[brain_name]
        observations = env_info.vector_observations  # Get observations for all agents

        while True:
            actions = maddpg.choose_action(observations)  # Choose actions for all agents
            clipped_actions = np.clip(actions, -1, 1)           # all actions between -1 and 1
            env_info = env.step(clipped_actions)[brain_name]

            observations_ = env_info.vector_observations
            rewards = env_info.rewards
            dones = env_info.local_done
            
            observations_vector = create_state_vector(observations)
            next_observations_vector = create_state_vector(observations_)

            score += np.sum(rewards)
            memory.store_transition(observations, observations_vector, actions, rewards, observations_,
                                next_observations_vector, dones)
            maddpg.learn(memory)

            observations = observations_
            global_timestep += 1
            
            if np.any(dones):
                break

        scores.append(score)
        avg_score = np.mean(scores[-100:])

        if avg_score > best_score:
            # Save the model for each agent
            for i, agent in enumerate(maddpg.agents):
                agent.save(f'agent_no{i}_{model_prefix}')

            best_score = avg_score
        
        if (episode % 100 == 0) and (episode > 0):
            print(f"Episode: {episode}, Best score: {best_score}, Avg score: {avg_score}, Global timestep: {global_timestep}")

    return scores

In [ ]:
scores = launcher(env, actions=action_size, episodes=4_000, model_prefix='model')
plot_scores(scores, output_file='agent_performance_plot.png')

In [ ]:
np.save('scores.npy', scores)

In [ ]:
env.close()